<a href="https://colab.research.google.com/github/BrandonLCC/Informe_Tecnico_FMY0100/blob/rama-modelos/Modelo_1_Regresi%C3%B3n_Lineal_Simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Importaciones y Dataset**

In [ ]:
#Las importaciones necesarias
import numpy as np
import pandas as pd # Estructura y manipulación de datos
import matplotlib.pyplot as plt # Graficos
from matplotlib import style # Si va
import seaborn as sns # diseño de Graficos

#---- Importaciónes de scikit learn ----

from sklearn.preprocessing import MinMaxScaler  # Con esta importacion, podemos normalizar los datos, para que la escala de los datos sean entre 0 y 1
from sklearn.preprocessing import LabelEncoder # Para transformar datos categoricos a formato numerico

from sklearn.linear_model import LinearRegression # con esta libreria podemos saber sobre problemas de clasificacion, regresion  y clustering () modelo lineal etc
from sklearn.model_selection import train_test_split

#---- Metricas de modelos de regresión ----

from sklearn.metrics import r2_score # solo lo ocuparemos para problemas de regresion o todo lo que sea numerico
from sklearn.metrics import mean_squared_error #solo para la parte numerica, con esto vamos a poder sumar y sacar el promedio del los datos o el error
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor # Importar DecisionTreeRegressor

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    name = fn

df = pd.read_csv(name, sep=";")

Saving Anexo ET_demo_round_traces_2022 (1).csv to Anexo ET_demo_round_traces_2022 (1).csv


<ipython-input-2-ee9ec52fcb6c>:8: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(name, sep=";")


# **2.Preparcion de datos**

## Respaldo del dataframe

In [ ]:
# Crear una variable como respaldo para poder arreglar el data set para poder utilizar despues para modelar
df_backup = df.copy()

## Limpieza de datos

In [ ]:
df_backup.drop(columns=['Unnamed: 0'], inplace=True)
# esta colmuna se elimnara ya que solamente contiene datos false lo que indica que ningun jugador ha presentado fallas en las partidas
df_backup.drop(columns=['AbnormalMatch'], inplace=True)
#esta colmuna FirstKillTime la eliminaremos ya que nos sirve para la hipotesis o preguntas
df_backup.drop(columns=['FirstKillTime'], inplace=True)
#Eliminad tiemplive y TravelledDistance podria no ser relevante para el analisis o hipotesis
df_backup.drop(columns=['TimeAlive'], inplace=True)
df_backup.drop(columns=['TravelledDistance'], inplace=True)

In [ ]:
# Como hay pocos nulos, eliminamos esas filas
df_backup.dropna(inplace=True)
# Verificar cantidad de nulos
print(df_backup.isnull().sum())

Map                            0
Team                           0
InternalTeamId                 0
MatchId                        0
RoundId                        0
RoundWinner                    0
MatchWinner                    0
Survived                       0
RLethalGrenadesThrown          0
RNonLethalGrenadesThrown       0
PrimaryAssaultRifle            0
PrimarySniperRifle             0
PrimaryHeavy                   0
PrimarySMG                     0
PrimaryPistol                  0
RoundKills                     0
RoundAssists                   0
RoundHeadshots                 0
RoundFlankKills                0
RoundStartingEquipmentValue    0
TeamStartingEquipmentValue     0
MatchKills                     0
MatchFlankKills                0
MatchAssists                   0
MatchHeadshots                 0
dtype: int64


### Eliminación de datos atipicos

In [ ]:
# Eliminando datos atipicos mayores a 28 de la culumna Asesinatos en la partida
df_backup.drop(df_backup[df_backup['MatchKills'] > 28].index, inplace=True)
df_backup.shape

(78992, 25)

In [ ]:
# Eliminando datos atipicos mayores a 8 de la culumna asistencias en la partida
df_backup.drop(df_backup[df_backup['MatchAssists'] > 8].index, inplace=True)
df_backup.shape

(78707, 25)

# **Trasformacion de datos**



* **Ver Valores Únicos en la Columna ""Team" y "Mapa"**

In [ ]:
df_backup.Team.unique()

array(['Terrorist', 'CounterTerrorist'], dtype=object)

## Uso de la técnica - label encoding


In [ ]:
label_encoder = LabelEncoder()
df_backup['Team'] = label_encoder.fit_transform(df_backup['Team'])
df_backup['Map'] = label_encoder.fit_transform(df_backup['Map'])

In [ ]:
df_backup['Map'].unique()

array([1, 3, 2, 0])

## Transformación de los datos Booleanos

In [ ]:
df_backup.RoundWinner = df_backup.RoundWinner.astype(bool)
df_backup.MatchWinner = df_backup.MatchWinner.astype(bool)
# convertir numeros
df_backup.RoundWinner.replace({True: 1, False: 0}, inplace=True)
df_backup.MatchWinner.replace({True: 1, False: 0}, inplace=True)
df_backup.Survived.replace({True: 1, False: 0}, inplace=True)

<ipython-input-13-4fe37939733c>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_backup.RoundWinner.replace({True: 1, False: 0}, inplace=True)
<ipython-input-13-4fe37939733c>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_backup.RoundWinner.replace({True: 1, False: 0}, inplace=True)
<ipython

In [ ]:
# Revisar estructura y tipos
df_backup.info()
# Vista previa de los primeros datos limpios
df_backup.head()

<class 'pandas.core.frame.DataFrame'>
Index: 78707 entries, 0 to 79156
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Map                          78707 non-null  int64  
 1   Team                         78707 non-null  int64  
 2   InternalTeamId               78707 non-null  int64  
 3   MatchId                      78707 non-null  int64  
 4   RoundId                      78707 non-null  int64  
 5   RoundWinner                  78707 non-null  int64  
 6   MatchWinner                  78707 non-null  int64  
 7   Survived                     78707 non-null  int64  
 8   RLethalGrenadesThrown        78707 non-null  int64  
 9   RNonLethalGrenadesThrown     78707 non-null  int64  
 10  PrimaryAssaultRifle          78707 non-null  float64
 11  PrimarySniperRifle           78707 non-null  float64
 12  PrimaryHeavy                 78707 non-null  float64
 13  PrimarySMG           

,Map,Team,InternalTeamId,MatchId,RoundId,RoundWinner,MatchWinner,Survived,RLethalGrenadesThrown,RNonLethalGrenadesThrown,...,RoundKills,RoundAssists,RoundHeadshots,RoundFlankKills,RoundStartingEquipmentValue,TeamStartingEquipmentValue,MatchKills,MatchFlankKills,MatchAssists,MatchHeadshots
0,1,1,1,4,1,1,1,0,0,4,...,0,0,0,0,750,4400,0,0,0,0
1,1,1,1,4,1,1,1,0,0,0,...,0,0,0,0,800,4400,0,0,0,0
2,1,1,1,4,1,1,1,0,0,0,...,0,0,0,0,1000,4400,0,0,0,0
3,1,1,1,4,1,1,1,0,0,0,...,0,0,0,0,850,4400,0,0,0,0
4,1,1,1,4,1,1,1,1,1,1,...,0,0,0,0,1000,4400,0,0,0,0



- limitar las rondas que sean mayores 30 rondas

In [ ]:
#  ajustar rondas que sean mayor a 30
df_backup = df_backup[(df_backup['RoundId'] >= 1) & (df_backup['RoundId'] <= 30)]

df_backup['RoundId'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

# **3. Modelo Regresión Lineal Simple**









entrenado modelo

In [ ]:
# Variables predictoras e independiente
#Definir las variables independientes (X) y la variable dependiente (y)
X = df_backup[['RoundStartingEquipmentValue']]
y = df_backup[['TeamStartingEquipmentValue']]

#Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X

,RoundStartingEquipmentValue
0,750
1,800
2,1000
3,850
4,1000
...,...
79152,5600
79153,5500
79154,4950
79155,4950


In [ ]:
y

,TeamStartingEquipmentValue
0,4400
1,4400
2,4400
3,4400
4,4400
...,...
79152,27100
79153,27100
79154,27100
79155,27100


In [ ]:
# Crear y entrenar el modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)


LinearRegression()

In [ ]:
# Predecir
y_pred = modelo.predict(X_test)

In [ ]:
# prompt:  mostrar resultado con mean_squared_error mean_absolute_error r2_score y root meam square error

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

# Imprimir resultados
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R2 Score: {r2}")


Mean Squared Error (MSE): 13814861.760270627
Mean Absolute Error (MAE): 2766.5792023794506
Root Mean Squared Error (RMSE): 3716.8349116244895
R2 Score: 0.8555792377899591


In [ ]:
# prompt:  puedes generar un gridseach cv validation para regresion linial simple usando arboles

# El código existente para la carga y preparación de datos ya está presente.
# Se asume que X_train, X_test, y_train, y_test ya están definidos.

# Crear un modelo de árbol de decisión para regresión
modelo_arbol = DecisionTreeRegressor(random_state=42) # Usamos random_state para reproducibilidad

# Definir los parámetros a tunear para el modelo de árbol de decisión
param_grid_arbol = {
    'max_depth': [None, 5, 10, 15, 20], # Profundidad máxima del árbol
    'min_samples_split': [2, 5, 10],    # Número mínimo de muestras requeridas para dividir un nodo interno
    'min_samples_leaf': [1, 2, 4]       # Número mínimo de muestras requeridas para ser un nodo hoja
}

# Crear el objeto GridSearchCV para el árbol de decisión
grid_search_arbol = GridSearchCV(estimator=modelo_arbol,
                                param_grid=param_grid_arbol,
                                cv=5, # Validación cruzada con 5 folds
                                scoring='neg_mean_squared_error', # Usamos MSE negativo para optimizar
                                n_jobs=-1) # Usa todos los núcleos disponibles

# Entrenar el modelo usando GridSearchCV
print("Iniciando GridSearchCV para DecisionTreeRegressor...")
grid_search_arbol.fit(X_train, y_train)
print("GridSearchCV completado.")

# Mostrar los mejores parámetros encontrados
print("\nMejores parámetros para DecisionTreeRegressor:", grid_search_arbol.best_params_)

# Mostrar la mejor puntuación (MSE negativo)
print("Mejor puntuación (MSE negativo) para DecisionTreeRegressor:", grid_search_arbol.best_score_)

# Obtener el mejor modelo de árbol encontrado
best_model_arbol = grid_search_arbol.best_estimator_

# Evaluar el mejor modelo de árbol en el conjunto de prueba
y_pred_arbol_grid = best_model_arbol.predict(X_test)

mse_arbol_grid = mean_squared_error(y_test, y_pred_arbol_grid)
mae_arbol_grid = mean_absolute_error(y_test, y_pred_arbol_grid)
r2_arbol_grid = r2_score(y_test, y_pred_arbol_grid)
rmse_arbol = np.sqrt(mse_arbol_grid)

print(f'\nResultados del modelo DecisionTreeRegressor (GridSearchCV):')
print(f'Mean Squared Error (MSE): {mse_arbol_grid}')
print(f'Mean Absolute Error (MAE): {mae_arbol_grid}')
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f'R-squared (R2): {r2_arbol_grid}')


Iniciando GridSearchCV para DecisionTreeRegressor...
GridSearchCV completado.

Mejores parámetros para DecisionTreeRegressor: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2}
Mejor puntuación (MSE negativo) para DecisionTreeRegressor: -9938093.67000499

Resultados del modelo DecisionTreeRegressor (GridSearchCV):
Mean Squared Error (MSE): 9730215.992013656
Mean Absolute Error (MAE): 2099.506853169133
Root Mean Squared Error (RMSE): 3716.8349116244895
R-squared (R2): 0.8982801830072448


#  **Resultados del Modelo**





Resumen del modelo


* R² Score 0.85  el modelo  indica una alta capacidad del modelo de la varianza

* MAE: 2.783
EL ERROR absoluto medio se desvian 2,783 unidades del valor

* MSE 14,216: El error cuadratico indica la presecencia de errores de predicion altos

* RMSE: 3716.83 este valor relativamente alto de RMSE sugiere que aún existen errores significativos en algunas predicciones.


Observacion:

* Como resultado de probar diferentes variables con el fin de observar distintos resultados, se identificaron los siguientes como los más destacados. Estos pueden considerarse entre los mejores modelos para las variables de regresión analizadas.


# **resultado del GridseahCV**


* Mean Squared Error (MSE): 10036814.610918598
* Mean Absolute Error (MAE): 2110.0773425250136
* R-squared (R2): 0.8943319363858644
* Root Mean Squared Error (RMSE): 3716.8349116244895

**Observacion:**

* r-cuadrado : 0.89 es mas alto su variancia en la variables'

* MSE : 10036814  disminuyo un poco en los errores de prediciones

* MAE : 2110  se puede observar  es un poco mas mejor  en la desviacion de los datos

* RMSE: 3716.83: como observacion no se encontro una mejora sigue siendo alto



